# Credit Prediction

## Context

### Credit risk
Credit Risk is the probable risk of loss resulting from a borrower's failure to repay a loan or meet contractual obligations. If a company offers credit to its client,then there is a risk that its clients may not pay their invoices.

### Types of Credit Risk
Good Risk: An investment that one believes is likely to be profitable. The term most often refers to a loan made to a creditworthy person or company. Good risks are considered exceptionally likely to be repaid.
Bad Risk: A loan that is unlikely to be repaid because of bad credit history, insufficient income, or some other reason. A bad risk increases the risk to the lender and the likelihood of default on the part of the borrower.

### Objective:
Based on the attributes, classify a person as good or bad credit risk.

### Dataset Description:
The dataset contains 1000 entries with 20 independent variables (7 numerical, 13 categorical) and 1 target variable prepared by Prof. Hofmann. In this dataset, each entry represents a person who takes a credit by a bank. Each person is classified as good or bad credit risks according to the set of attributes.The attributes are:

### Features
* Status of existing checking account, in Deutsche Mark.
* Duration in months
* Credit history (credits taken, paid back duly, delays, critical accounts)
* Purpose of the credit (car, television,...)
* Credit amount
* Status of savings account/bonds, in Deutsche Mark.
* Present employment, in number of years.
* Installment rate in percentage of disposable income
* Personal status (married, single,...) and sex
* Other debtors / guarantors
* Present residence since X years
* Property (e.g. real estate)
* Age in years
* Other installment plans (banks, stores)
* Housing (rent, own,...)
* Number of existing credits at this bank
* Job
* Number of people being liable to provide maintenance for
* Telephone (yes,no)
* Foreign worker (yes,no)

### Target
* Grant credit `good` or denied credit `bad`

In [ ]:
from xautoml.util.datasets import openml_task

X_train, y_train = openml_task(31, 0, train=True)
X_train

## Start the Model Building

You load the data set in an AutoML tool you have found on the internet, to create a predictive model. After starting the optimization, the AutoML tool tests various possible models and evaluates how good each candidate is. In the meantime you have to wait for the program to finish its optimization.

In [ ]:
import pickle
from dswizard.optimizers.bandit_learners import PseudoBandit
from dswizard.optimizers.structure_generators.mcts import TransferLearning, MCTS
from dswizard.optimizers.config_generators import Hyperopt
from dswizard.core.master import Master
from dswizard.core.model import Dataset
from dswizard.util import util

util.setup_logging('/opt/xautoml/dswizard/output/mcts/log.txt')

ds = Dataset(X_train.values, y_train.values, task=31, metric='accuracy', feature_names=X_train.columns)
master = Master(
    ds=ds,
    working_directory='/opt/xautoml/dswizard/output/mcts/',
    n_workers=1,
    model='rf_complete.pkl',

    wallclock_limit=600,
    cutoff=10,
    pre_sample=False,

    config_generator_class=Hyperopt,
    structure_generator_class=MCTS,
    structure_generator_kwargs={'policy': TransferLearning},
    bandit_learner_class=PseudoBandit
)

pipeline, run_history, ensemble = master.optimize()

with open('/opt/xautoml/dswizard/output/mcts/dswizard.pkl', 'wb') as f:
    pickle.dump((run_history, ensemble), f)


In [ ]:
import pickle
import joblib

with open(f'/opt/xautoml/dswizard/output/mcts/dswizard.pkl', 'rb') as f:
    run_history, ensemble = pickle.load(f)

with open(f'/opt/xautoml/dswizard/output/mcts/incumbent.pkl', 'rb') as f:
    pipeline = joblib.load(f)

After waiting for 10 minutes, you are presented with the following results:

### The score of the Final Model

Internally, the AutoML tool uses a measure to determine how good a candidate is, for example the number of correct predictions (accuracy). After the optimization, you want to test how good the model actually is before using it with patients. Therefore, you have hidden a part of the data set which you will now use to test how good the best model actually is:

In [ ]:
from sklearn.metrics import accuracy_score

X_test, y_test = X_train, y_train = openml_task(31, 0, test=True)

predictions = ensemble.predict(X_test.values)
accuracy_score(y_test, predictions)

Meaning, that the generated model is able to predict that many new patients, it has never seen before, correctly.


### View the Models found by dswizard

Besides the raw performance, the tool also tells you which the best models are

In [ ]:
pipeline

With this information you are good to go and can decide if you actually want to use the generated model.

## Load the Same Results in XAutoML

In [ ]:
from xautoml.main import XAutoML
from xautoml.adapter import import_dswizard
from xautoml.util.datasets import openml_task
import pickle

with open(f'/opt/xautoml/dswizard/output/mcts/dswizard.pkl', 'rb') as f:
    run_history, ensemble = pickle.load(f)

X_test, y_test = openml_task(31, 0, test=True)

rh = import_dswizard(run_history, ensemble)
main = XAutoML(rh, X_test, y_test)
main